In [1]:
import os

# List the current directory
#print(os.listdir(.))

In [ ]:
# Ask the user to select a digit in the range 0 to 8
k = int(input("Please select a digit in the range 0 to 8: "))

# Check if the digit is within the valid range
if 0 <= k <= 8:
    print(f"will be running in k{k}")
else:
    print("Invalid selection. Please select a digit between 0 and 8.")
    exit(1)

from pathlib import Path

wd = Path(f"/home/rfriedma/src/k{k}/ceph/build")
os.chdir(wd)

In [ ]:
!ls -l
!bash -c MGR=0 ls -l

In [ ]:
%%bash
scrtch=to_"`date +%d_%H%M`"
echo $scrtch

MDS=0 MGR=1 OSD=4 MON=1 ../src/vstart.sh -n --msgr2 -X --memstore -o "memstore_device_bytes=68435456" -o "osd_op_queue=wpq"
sleep 2
bin/ceph -s

bin/ceph tell osd.* config set debug_osd 20/20
#bin/ceph osd set nodown

bin/ceph config set global osd_pool_default_pg_autoscale_mode off
sleep 2

#PL1 is of size 3

bin/ceph osd pool create pl1 4 4
#bin/ceph osd pool autoscale-status
sleep 1
bin/ceph osd pool set pl1 size 3
bin/ceph osd pool set pl1 min_size 3
bin/ceph osd pool set pl1 pg_autoscale_mode off
bin/ceph osd pool stats
bin/ceph osd pool set pl1 noscrub 1
sleep 2

bin/rados bench -p pl1 -t 1 1 write -b 4096 --max-objects 8  --no-cleanup; 
bin/rados bench -p pl1 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee


In [6]:
%%bash
# common scrub configs
bin/ceph tell osd.* config set osd_blocked_scrub_grace_period 20
bin/ceph tell osd.* config set osd_stats_update_period_scrubbing 2
bin/ceph tell osd.* config set osd_stats_update_period_not_scrubbing 3
bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0
bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0
bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0

bin/ceph tell mgr.$(bin/ceph mgr services | jq -r .mgr) config set mgr_stats_period 2


In [7]:
%%bash
# set scrub parameters to guarantee slow scrub
bin/ceph tell osd.* config set osd_scrub_sleep "3.0"
bin/ceph tell osd.* config set osd_max_scrubs 1
bin/ceph tell osd.* config set osd_scrub_chunk_max 5
bin/ceph tell osd.* config set osd_shallow_scrub_chunk_max 5


In [ ]:
raise SystemExit("Stop here")

# Termination


In [ ]:
%%bash
../src/stop.sh
sleep 4
../src/stop.sh
